<a href="https://colab.research.google.com/github/LC1332/Chat-Haruhi-Suzumiya/blob/main/kyon_generator/test_kyon_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chat凉宫春日 Chat-Haruhi-Suzumiya

[![Code License](https://img.shields.io/badge/Code%20License-Apache_2.0-green.svg)]()
[![Data License](https://img.shields.io/badge/Data%20License-CC%20By%20NC%204.0-red.svg)]()
[![Huggingface Gradio](https://img.shields.io/static/v1?label=Demo&message=Huggingface%20Gradio&color=orange)](https://huggingface.co/spaces/silk-road/ChatHaruhi)


本脚本是李鲁鲁开发的KyonGenerator的测试工具，后者由

李鲁鲁，闫晨曦, 包同学, 睡觉鱼等共同开发

---

**Chat凉宫春日**是模仿凉宫春日等一系列动漫人物，使用近似语气、个性和剧情聊天的语言模型，

<details>
  <summary> 本项目由李鲁鲁，冷子昂，闫晨曦，封小洋，scixing，沈骏一，Aria Fei, 米唯实, 吴平宇, 贾曜恺等开发。 </summary>

李鲁鲁发起了项目，并完成了最早的版本，在多个微信群实现了测试。完成了训练数据自动生成对话部分，设计了整体的路线，并撰写报告。

冷子昂负责了每一个阶段的Gradio开发，以及每个部分的功能整合和架构设计。

闫晨曦一开始将李鲁鲁的notebook重构为app.py，参与了WebUI的embedding部分重构整合。

封小洋进行了中文转日文模型的选型，完成了针对台词抽取图片的工具。整合了声纹识别。即将继续参加台本工具的开发。

scixing实践了VITS语音，完成了台词对应的语音抽取，实现了第一个版本的声纹分类。

沈骏一实现了使用ChatGLM2 finetune实验

Aria(Yaying Fei)对接了whisper到台本工具。即将继续参加台本工具的开发。

米唯实实现了Chat哆啦A梦的分支版本

吴平宇部署了ChatGLM2的训练程序，并提供了训练的计算资源。

贾曜恺开发了一个Vue前端实现方案

</details>

In [1]:
!git clone https://github.com/LC1332/Chat-Haruhi-Suzumiya

Cloning into 'Chat-Haruhi-Suzumiya'...
remote: Enumerating objects: 5240, done.
remote: Counting objects: 100% (1543/1543), done.
remote: Compressing objects: 100% (1161/1161), done.
remote: Total 5240 (delta 389), reused 1494 (delta 369), pack-reused 3697
Receiving objects: 100% (5240/5240), 103.28 MiB | 17.30 MiB/s, done.
Resolving deltas: 100% (1817/1817), done.
Updating files: 100% (2211/2211), done.


In [12]:
!pip install -q openai langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.0 MB/s eta 0:00:00


In [2]:
%cd /content/Chat-Haruhi-Suzumiya/kyon_generator/
!ls

/content/Chat-Haruhi-Suzumiya/kyon_generator
chat2dialogue.py	   data			  story2chat.py
ChatGPT_for_generation.py  dialogue2chat.py	  synthesis_chat_method_foo.py
chatlog2dialogue.ipynb	   dialogue2embedding.py  synthesis_chat.py
chatlog2dialogue.py	   dialogue2jsonl.py	  train.py


In [3]:
!ls data

Haruhi-ContinuousGenerate_Dialogues.jsonl  Haruhi-Lulu_Dialogues.jsonl
Haruhi_first_merge_res.jsonl


In [4]:
!mkdir output

## gradio前端记录的历史数据到dialogue数据
chatlog2dialogue.py gradio前端记录的历史数据到dialogue数据的转化



In [5]:
!python chatlog2dialogue.py -input ./data/Haruhi-Lulu_Dialogues.jsonl\
                           -output ./output/Haruhi_Dialogues_from_history.jsonl


## story2chat 台本数据到一句话数据的转换

Haruhi为例的话语料在

/content/Chat-Haruhi-Suzumiya/characters/haruhi/texts

In [7]:
!python story2chat.py -story_folder "/content/Chat-Haruhi-Suzumiya/characters/haruhi/texts" \
                     -output ./output/chat_from_story.jsonl \
                     -role "春日" \
                     -other_names 凉宫 凉宫春日

## dialogue2chat 将连续对话批量抽取为一句话

In [8]:
!python dialogue2chat.py -input ./data/Haruhi-Lulu_Dialogues.jsonl\
                     -output ./output/chat_from_dialogue.jsonl\
                     -role "春日"\
                     -other_names 凉宫 凉宫春日

## synthesis_chats 将一些第一句话的文件，增广为更多的第一句话

这里需要设置OpenAI Key

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "sk-KP8Zco"

TODO: 这里Haruhi_first_merge_res.jsonl本来就是synthesized的，

最好替换为之前的输出 ./output/chat_from_dialogue.jsonl

In [ ]:
!python synthesis_chat.py -input ./data/Haruhi_first_merge_res.jsonl\
                         -output ./output/chat_augmented_by_foo.jsonl \
                         -method foo

## chat2dialogue 通过inference引擎，将第一句话输入到里面去要出更多的对话

In [14]:
!python chat2dialogue.py -input_chat ./data/Haruhi_first_merge_res.jsonl\
                        -output_dialogue ./output/synthesized_dialogue.jsonl \
                        -config config.ini \
                        -role_name "春日"\
                        -other_names 凉宫 凉宫春日

Traceback (most recent call last):
  File "/content/Chat-Haruhi-Suzumiya/kyon_generator/chat2dialogue.py", line 18, in <module>
    from ChatGPT_for_generation import ChatGPT
  File "/content/Chat-Haruhi-Suzumiya/kyon_generator/ChatGPT_for_generation.py", line 11, in <module>
    import utils
ModuleNotFoundError: No module named 'utils'
